# advanced contexts

We already introduced synthesizable contexts in the form of `std.sequential` and `std.concurrent`. Since these are part of the `std` module they are not magic and receive no special treatment by the compiler. This section covers the underlying CoHDL functions and demonstrates how custom contexts can be defined.

In [1]:
from cohdl import Entity, Port, Unsigned, Bit
from cohdl import std
import cohdl


class Example(Entity):
    clk = Port.input(Bit)
    reset = Port.input(Bit)

    data_in = Port.input(Unsigned[8])
    data_valid = Port.input(Bit)
    data_ready = Port.output(Bit, default=False)

    sum = Port.output(Unsigned[8], default=0)

    def architecture(self):
        @std.sequential(std.Clock(self.clk), std.Reset(self.reset))
        async def proc():
            await self.data_valid
            self.sum <<= self.sum + self.data_in
            self.data_ready ^= True

print(std.VhdlCompiler.to_string(Example))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity Example is
  port (
    clk : in std_logic;
    reset : in std_logic;
    data_in : in unsigned(7 downto 0);
    data_valid : in std_logic;
    data_ready : out std_logic;
    sum : out unsigned(7 downto 0)
    );
end Example;


architecture arch_Example of Example is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_data_ready : std_logic := '0';
  signal buffer_sum : unsigned(7 downto 0) := unsigned(std_logic_vector'("00000000"));
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  data_ready <= buffer_data_ready;
  sum <= buffer_sum;
  

  proc: process(clk)
    variable temp : unsigned(7 downto 0);
  begin
    if rising_edge(clk) then
      if reset = '1' then
        buffer_sum <= unsigned(std_logic_vector'("00000000"));
        buffer_data_ready <

`std.sequential` is built around these CoHDL builtins

* `sensitivity`
* `rising_edge`/`falling_edge`
* `reset_context`
* `reset_pushed`
* `coroutine_step`

the next code block shows how they can replace `std.sequential` to produce the same VHDL as before.

In [2]:
class Example(Entity):
    clk = Port.input(Bit)
    reset = Port.input(Bit)

    data_in = Port.input(Unsigned[8])
    data_valid = Port.input(Bit)
    data_ready = Port.output(Bit, default=False)

    sum = Port.output(Unsigned[8], default=0)

    def architecture(self):
        # cohdl.sequential_context only operates on normal functions
        # not coroutines. State machines are actually inferred by
        # cohdl.coroutine_step.
        async def proc():
            await self.data_valid
            self.sum <<= self.sum + self.data_in
            self.data_ready ^= True
        
        # std.sequential wraps coroutines in normal functions
        # and builds clock/reset code around it
        @cohdl.sequential_context
        def wrapper():
            # When the compiler encounters a call to cohdl.sensitivity.list
            # it sets the sensitivity list of the generated VHDL process.
            # Otherwise the sensitivity list will be filled with all
            # signals read in this context.
            cohdl.sensitivity.list(self.clk, self.reset)

            # std.sequential() adds if-rising_edge or if-falling_edge
            # depending on the given clock argument
            if cohdl.rising_edge(self.clk):
                if self.reset:
                    # the compiler replaces each call to cohdl.reset_context
                    # with default assignments to all Signals/Variables driven
                    # by this context
                    cohdl.reset_context()
                else:
                    # cohdl.reset_pushed works like cohdl.reset_context
                    # but only pushed (^= operator) Signals are reset
                    cohdl.reset_pushed()

                    # cohdl.coroutine_step is replaced with the state machine
                    # inferred from the given coroutine
                    cohdl.coroutine_step(proc())

print(std.VhdlCompiler.to_string(Example))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity Example is
  port (
    clk : in std_logic;
    reset : in std_logic;
    data_in : in unsigned(7 downto 0);
    data_valid : in std_logic;
    data_ready : out std_logic;
    sum : out unsigned(7 downto 0)
    );
end Example;


architecture arch_Example of Example is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_data_ready : std_logic := '0';
  signal buffer_sum : unsigned(7 downto 0) := unsigned(std_logic_vector'("00000000"));
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  data_ready <= buffer_data_ready;
  sum <= buffer_sum;
  

  wrapper: process(clk, reset)
    variable temp : boolean;
    variable temp_1 : unsigned(7 downto 0);
  begin
    if rising_edge(clk) then
      temp := reset = '1';
      if temp then
        buffer_sum <= unsigne

With that knowledge we can define our own striped down version of `std.sequential()` and apply it to coroutines.

`std.sequential()` has a few more configuration parameters

* rising/falling edge clocks
* sync/async/high-active/low-active reset
* determines if the argument is a normal function or coroutine

but is otherwise implemented like the simple example.

In [3]:
def simple_sequential(clk, reset):
    def wrapper(coro):

        @cohdl.sequential_context
        def sequential_fn():
            cohdl.sensitivity.list(clk, reset)

            if cohdl.rising_edge(clk):
                if reset:
                    cohdl.reset_context()
                else:
                    cohdl.reset_pushed()
                    cohdl.coroutine_step(coro())
    
    return wrapper
            

class Example(Entity):
    clk = Port.input(Bit)
    reset = Port.input(Bit)

    data_in = Port.input(Unsigned[8])
    data_valid = Port.input(Bit)
    data_ready = Port.output(Bit, default=False)

    sum = Port.output(Unsigned[8], default=0)

    def architecture(self):
        @simple_sequential(self.clk, self.reset)
        async def proc():
            await self.data_valid
            self.sum <<= self.sum + self.data_in
            self.data_ready ^= True

print(std.VhdlCompiler.to_string(Example))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity Example is
  port (
    clk : in std_logic;
    reset : in std_logic;
    data_in : in unsigned(7 downto 0);
    data_valid : in std_logic;
    data_ready : out std_logic;
    sum : out unsigned(7 downto 0)
    );
end Example;


architecture arch_Example of Example is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_data_ready : std_logic := '0';
  signal buffer_sum : unsigned(7 downto 0) := unsigned(std_logic_vector'("00000000"));
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  data_ready <= buffer_data_ready;
  sum <= buffer_sum;
  

  sequential_fn: process(clk, reset)
    variable temp : boolean;
    variable temp_1 : unsigned(7 downto 0);
  begin
    if rising_edge(clk) then
      temp := reset = '1';
      if temp then
        buffer_sum <= u

That explains the difference between `std.sequential` and the fundamental `cohdl.sequential_context`. `std.concurrent` only exists for symmetry with `std.sequential` and is otherwise equivalent to `cohdl.concurrent_context`.

`std.sequential` should cover more than 95% of all use cases. However sometimes the added flexibility of the underlying API is needed. For example the code below defines a design where the state machine is only advanced when an enable signal is set.

In [4]:
def enabled_sequential(clk, reset, enable):
    def wrapper(coro):

        @cohdl.sequential_context
        def sequential_fn():
            cohdl.sensitivity.list(clk, reset)

            if cohdl.rising_edge(clk):
                if reset:
                    cohdl.reset_context()
                else:
                    cohdl.reset_pushed()

                    if enable:
                        # only advance the state machine
                        # when enable is true
                        cohdl.coroutine_step(coro())
    
    return wrapper

class Example(Entity):
    clk = Port.input(Bit)
    reset = Port.input(Bit)
    enable = Port.input(Bit)

    data_in = Port.input(Unsigned[8])
    data_valid = Port.input(Bit)
    data_ready = Port.output(Bit, default=False)

    sum = Port.output(Unsigned[8], default=0)

    def architecture(self):
        @enabled_sequential(self.clk, self.reset, self.enable)
        async def proc():
            await self.data_valid
            self.sum <<= self.sum + self.data_in
            self.data_ready ^= True

print(std.VhdlCompiler.to_string(Example))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity Example is
  port (
    clk : in std_logic;
    reset : in std_logic;
    enable : in std_logic;
    data_in : in unsigned(7 downto 0);
    data_valid : in std_logic;
    data_ready : out std_logic;
    sum : out unsigned(7 downto 0)
    );
end Example;


architecture arch_Example of Example is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_data_ready : std_logic := '0';
  signal buffer_sum : unsigned(7 downto 0) := unsigned(std_logic_vector'("00000000"));
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  data_ready <= buffer_data_ready;
  sum <= buffer_sum;
  

  sequential_fn: process(clk, reset)
    variable temp : boolean;
    variable temp_1 : boolean;
    variable temp_2 : unsigned(7 downto 0);
  begin
    if rising_edge(clk) then
      temp 